# Scorecard:

### Import libraries

In [427]:
import pandas as pd
import numpy as np
import os
import math

In [428]:
from sklearn.model_selection import train_test_split

In [429]:
import statsmodels.api as sm

### Set a project path

In [430]:
project_path = r"C:\Data D\Projects\Python\Experian Project"

In [431]:
%run "C:\Data D\Projects\Python\Experian Project\data_dictionary.py"

### Import the data set

In [432]:
data_sample = os.path.join(project_path, r"BankCaseStudyData.csv")
mycs_data = pd.read_csv(data_sample)

### Inspect and prepare the data

In [433]:
mycs_data.shape

(24859, 33)

In [434]:
mycs_data.columns

Index(['Account_Number', 'Account_Type', 'Final_Decision',
       'Current_Delinquency_status', 'Application_Date', 'Application_Score',
       'Cheque_Card_Flag', 'Existing_Customer_Flag', 'Gross_Annual_Income',
       'Home_Telephone_Number', 'Insurance_Required', 'Loan_Amount',
       'Loan_Payment_Frequency', 'Loan_Payment_Method', 'Marital_Status',
       'Number_of_Dependants', 'Number_of_Payments', 'Occupation_Code',
       'Promotion_Type', 'Residential_Status', 'Time_at_Address',
       'Time_in_Employment', 'Time_with_Bank', 'Weight_Factor', 'GB_Flag',
       'Age_of_Applicant', 'Application_Month', 'Bureau_Score',
       'SP_ER_Reference', 'SP_Number_Of_Searches_L6M', 'SP_Number_of_CCJs',
       'loan_to_income', 'split'],
      dtype='object')

In [435]:
mycs_data[pd.isna(mycs_data['Current_Delinquency_status'])].shape

(6044, 33)

In [436]:
mycs_data['Current_Delinquency_status'].value_counts(dropna=False).sort_index()

0.0    7606
1.0    9232
2.0     847
3.0      14
4.0     502
5.0     345
6.0     269
NaN    6044
Name: Current_Delinquency_status, dtype: int64

In [437]:
mycs_data[mycs_data.duplicated('Account_Number')==True].shape #check for duplication in Account number

(0, 33)

In [438]:
data_dictionary(mycs_data) #Apply the data dictionary

In [439]:
mycs_data.head(10)

,Account_Number,Account_Type,Final_Decision,Current_Delinquency_status,Application_Date,Application_Score,Cheque_Card_Flag,Existing_Customer_Flag,Gross_Annual_Income,Home_Telephone_Number,...,Weight_Factor,GB_Flag,Age_of_Applicant,Application_Month,Bureau_Score,SP_ER_Reference,SP_Number_Of_Searches_L6M,SP_Number_of_CCJs,loan_to_income,split
0,10730734532,Fixed Loan,Accept,NaN,20061206,965,Yes,Yes,12000,No,...,2.0,NTU,28,200612,1009,Confirmed as previous occupant,0,0,15.50,Development
1,10803550208,Variable Loan,Decline,NaN,20060928,720,No,Yes,10015,Yes,...,2.0,Rejects,36,200609,784,Not confirmed,0,0,29.96,Development
2,10769083290,Fixed Loan,Accept,0.0,20060721,975,Yes,No,11000,Yes,...,2.0,Good,48,200607,940,Confirmed,2,0,45.45,Development
3,10072636331,Fixed Loan,Accept,1.0,20060529,960,Yes,No,16500,Yes,...,2.0,Good,41,200605,902,Confirmed,1,0,31.82,Development
4,10737329597,Fixed Loan,Accept,0.0,20060718,980,Yes,Yes,60000,Yes,...,2.0,Good,37,200607,1013,Confirmed,7,0,16.67,Development
5,10464533150,Fixed Loan,Accept,1.0,20061109,920,No,No,22300,Yes,...,2.0,Good,47,200611,945,Confirmed,1,0,17.94,Development
6,10238114351,Fixed Loan,Accept,0.0,20060328,985,Yes,No,12444,Yes,...,2.0,Good,38,200603,1002,Confirmed,1,0,-9999998.00,Development
7,10639250159,Variable Loan,Decline,NaN,20070122,695,No,No,9360,Yes,...,2.0,Rejects,19,200701,787,Not confirmed,0,0,26.71,Development
8,10686407596,Variable Loan,Accept,2.0,20060714,945,Yes,Yes,9000,Yes,...,2.0,Indeterminate,25,200607,986,Confirmed as previous occupant,2,0,17.78,Development
9,10130851809,Fixed Loan,Accept,0.0,20061017,965,No,No,12000,Yes,...,2.0,Good,34,200610,844,Confirmed,3,0,12.65,Development


### Derive Target Variable

In [440]:
mycs_data['Current_Delinquency_status'].value_counts(dropna=False).sort_index()

0.0    7606
1.0    9232
2.0     847
3.0      14
4.0     502
5.0     345
6.0     269
NaN    6044
Name: Current_Delinquency_status, dtype: int64

In [441]:
# create a crosstab between 'Final_Decision' and 'Time_with_Bank' to see the Accept/Decline rate
pd.crosstab(mycs_data['Final_Decision'],mycs_data['Current_Delinquency_status']) 

Current_Delinquency_status,0.0,1.0,2.0,3.0,4.0,5.0,6.0
Final_Decision,,,,,,,
Accept,7606,9232,847,14,502,345,269


In [442]:
pd.crosstab(mycs_data['Final_Decision'],mycs_data['Current_Delinquency_status'].fillna(-1), margins="Total")

Current_Delinquency_status,-1.0,0.0,1.0,2.0,3.0,4.0,5.0,6.0,All
Final_Decision,,,,,,,,,
Accept,3447,7606,9232,847,14,502,345,269,22262
Decline,2597,0,0,0,0,0,0,0,2597
All,6044,7606,9232,847,14,502,345,269,24859


In [443]:
# Setting Current Deliquency status as the target variable
mycs_data['target'] = (
        np.select(
        condlist=[((mycs_data['Current_Delinquency_status'].isna()) & (mycs_data['Final_Decision'] == 'Accept')),
                  ((mycs_data['Current_Delinquency_status'].isna()) & (mycs_data['Final_Decision'] == 'Decline')),
                  mycs_data['Current_Delinquency_status'] < 2,
                  mycs_data['Current_Delinquency_status'] == 2,
                  mycs_data['Current_Delinquency_status']  > 2,
                  ],
        choicelist=['NTU', 'Rejects', 'Good', 'Indeterminate', 'Bad']))

In [444]:
mycs_data['target'].value_counts(dropna=False) 

Good             16838
NTU               3447
Rejects           2597
Bad               1130
Indeterminate      847
Name: target, dtype: int64

In [445]:
print(round(1119/(16726+1119)*100,2), "%") # Calculating the bad rate %

6.27 %


In [446]:
mycs_data = mycs_data[((mycs_data['target'] == 'Good') | (mycs_data['target'] == 'Bad'))] #We'll work with only Good and Bad Targets

In [447]:
mycs_data['num_target'] = np.where(mycs_data['target'] =="Good", 1, 0) # Assigning values 1 for Good, and 0 for Bad 

## Categorical Variables via log odds:

Here I've tried to use the log odds to bin the categorical variables. In the end I decided not to use it, because I was unsure how to implement it further into the scorecard.

In [448]:
mycs_data['Account_Type'].value_counts(dropna=False).sort_index()

Fixed Loan       12801
Variable Loan     5167
Name: Account_Type, dtype: int64

In [449]:
fixed_loan_probability = (16828/24859)*100

In [450]:
fixed_loan_probability

67.69379299247757

In [451]:
variable_loan_probability = (8031/24859)*100

In [452]:
variable_loan_probability

32.306207007522424

In [453]:
fixed_loan_logodds = math.log(67.69379299247757/32.306207007522424)

In [454]:
fixed_loan_logodds

0.7397351125219493

In [455]:
variable_loan_logodds = math.log(32.306207007522424/67.69379299247757)

In [456]:
variable_loan_logodds

-0.7397351125219493

In [457]:
mycs_data['Cheque_Card_Flag'].value_counts(dropna=False).sort_index()

No      5731
Yes    12235
NaN        2
Name: Cheque_Card_Flag, dtype: int64

In [458]:
No_probability = (9364/24859)*100

In [459]:
No_probability

37.66845005832897

In [460]:
math.log(37.66845005832897/(100-37.66845005832897))

-0.5036448416110475

In [461]:
Yes_probability = (15492/24859)*100

In [462]:
Yes_probability

62.319481877790736

In [463]:
math.log(62.319481877790736/(100-62.319481877790736))

0.503130887425512

In [464]:
Nan_probability = (3/24859)*100

In [465]:
Nan_probability

0.012068063880284807

In [466]:
math.log(0.012068063880284807/(100-0.012068063880284807))

-9.022242162404774

In [467]:
math.log(((3/24859)*100)/(100-(3/24859)*100))

-9.022242162404774

# Classing

## Loan to Income

In [468]:
mycs_data['loan_to_income'].value_counts(dropna=False)

-9999997.00    910
-9999998.00    724
 20.00         293
 25.00         251
 33.33         237
              ... 
 36.14           1
 4.06            1
 8.71            1
 273.08          1
 16.74           1
Name: loan_to_income, Length: 5028, dtype: int64

In [469]:
mycs_data['loan_to_income'].describe()

count    1.796800e+04
mean    -9.093333e+05
std      2.875328e+06
min     -9.999998e+06
25%      1.083000e+01
50%      2.317000e+01
75%      4.231000e+01
max      1.160294e+04
Name: loan_to_income, dtype: float64

In [470]:
mycs_data['loan_to_income_classing'] = pd.cut(mycs_data['loan_to_income'],[-9999998.00,-9999997.00, 0, 1450, 2900, 4350, 5800, 7250, 8700, np.inf],right=False)
# We specify the borders of the ranges

In [471]:
mycs_data['loan_to_income_classing'].value_counts(dropna=False).sort_index() # Check to see how many of the numbers fall into each of the ranges

[-9999998.0, -9999997.0)      724
[-9999997.0, 0.0)             910
[0.0, 1450.0)               16296
[1450.0, 2900.0)               31
[2900.0, 4350.0)                0
[4350.0, 5800.0)                2
[5800.0, 7250.0)                0
[7250.0, 8700.0)                1
[8700.0, inf)                   4
Name: loan_to_income_classing, dtype: int64

In [472]:
mycs_data['loan_to_income_classing'] = pd.cut(mycs_data['loan_to_income'],[-9999998.00,-9999997.00, 0, 362, 725, 1087, 1450, np.inf],right=False)
# Since almost all of the numbers fall into the range (0, 1450) we'll need to narrow it down 

In [473]:
mycs_data['loan_to_income_classing'].value_counts(dropna=False).sort_index() # Check to see how the ranges changed

[-9999998.0, -9999997.0)      724
[-9999997.0, 0.0)             910
[0.0, 362.0)                15686
[362.0, 725.0)                438
[725.0, 1087.0)               131
[1087.0, 1450.0)               41
[1450.0, inf)                  38
Name: loan_to_income_classing, dtype: int64

In [474]:
mycs_data['loan_to_income_classing'] = pd.cut(mycs_data['loan_to_income'],[-9999998.00,-9999997.00, 0, 100, 200, 300, 400, 500, 600, 700, np.inf],right=False)
# We narrow it down again

In [475]:
mycs_data['loan_to_income_classing'].value_counts(dropna=False).sort_index() # Check the ranges again

[-9999998.0, -9999997.0)      724
[-9999997.0, 0.0)             910
[0.0, 100.0)                14734
[100.0, 200.0)                544
[200.0, 300.0)                264
[300.0, 400.0)                223
[400.0, 500.0)                150
[500.0, 600.0)                121
[600.0, 700.0)                 66
[700.0, inf)                  232
Name: loan_to_income_classing, dtype: int64

In [476]:
mycs_data['loan_to_income_classing'] = pd.cut(mycs_data['loan_to_income'],[-9999998.00,-9999997.00, 0, 20, 40, 60, 80, 100, np.inf],right=False)
#Further narrowing shows...

In [477]:
mycs_data['loan_to_income_classing'].value_counts(dropna=False).sort_index() # Check the ranges again

[-9999998.0, -9999997.0)     724
[-9999997.0, 0.0)            910
[0.0, 20.0)                 6130
[20.0, 40.0)                5231
[40.0, 60.0)                2279
[60.0, 80.0)                 774
[80.0, 100.0)                320
[100.0, inf)                1600
Name: loan_to_income_classing, dtype: int64

In [478]:
mycs_data.groupby(['loan_to_income_classing'])['num_target'].mean()

loan_to_income_classing
[-9999998.0, -9999997.0)    0.980663
[-9999997.0, 0.0)           0.947253
[0.0, 20.0)                 0.944698
[20.0, 40.0)                0.935385
[40.0, 60.0)                0.935059
[60.0, 80.0)                0.918605
[80.0, 100.0)               0.921875
[100.0, inf)                0.903125
Name: num_target, dtype: float64

## Bureau Score

In [479]:
mycs_data['Bureau_Score'].value_counts(dropna=False).sort_index() # check the contents of the 'Bureau Score' column

636      1
643      1
647      3
653      1
657      1
        ..
1052     3
1053     7
1054    37
1056    39
1058    51
Name: Bureau_Score, Length: 357, dtype: int64

In [480]:
mycs_data['Bureau_Score'].describe() 

count    17968.000000
mean       933.011520
std         68.032139
min        636.000000
25%        888.000000
50%        941.000000
75%        983.000000
max       1058.000000
Name: Bureau_Score, dtype: float64

In [481]:
mycs_data['Bureau_Score_classing'] = pd.cut(mycs_data['Bureau_Score'],[-np.inf, 610, 1058, np.inf],right=False) #set a range

In [482]:
mycs_data['Bureau_Score_classing'].value_counts(dropna=False).sort_index() # check the elements

[-inf, 610.0)          0
[610.0, 1058.0)    17917
[1058.0, inf)         51
Name: Bureau_Score_classing, dtype: int64

In [483]:
mycs_data['Bureau_Score_classing'] = pd.cut(mycs_data['Bureau_Score'],[-np.inf, 710, 810, 910, 1010, np.inf],right=False) #set a new range

In [484]:
mycs_data['Bureau_Score_classing'].value_counts(dropna=False).sort_index() # check the elements again

[-inf, 710.0)        37
[710.0, 810.0)      724
[810.0, 910.0)     5054
[910.0, 1010.0)    9805
[1010.0, inf)      2348
Name: Bureau_Score_classing, dtype: int64

In [485]:
mycs_data.groupby(['Bureau_Score_classing'])['num_target'].mean()

Bureau_Score_classing
[-inf, 710.0)      0.648649
[710.0, 810.0)     0.798343
[810.0, 910.0)     0.896913
[910.0, 1010.0)    0.958695
[1010.0, inf)      0.980835
Name: num_target, dtype: float64

## Occupation Code

In [486]:
mycs_data['Occupation_Code'].value_counts()

Employee         8266
Self-employed    6609
Pensioner        2680
Other             413
Name: Occupation_Code, dtype: int64

In [487]:
# if Occupation_Code = M, then Occupation_Code_classing = M, else Occupation_Code_classing = 'O|P|B'

mycs_data['Occupation_Code_classing'] = np.where(mycs_data['Occupation_Code'] =="Employee", 'M', 'O|P|B')
mycs_data.groupby(['Occupation_Code_classing'])['num_target'].mean() # check the good score

Occupation_Code_classing
M        0.921849
O|P|B    0.950113
Name: num_target, dtype: float64

In [488]:
# Since I want to have the better score on the single variable, I'll switch 'Employee' for 'Self-employed' 
# if Occupation_Code = B, then Occupation_Code_classing = B, else Occupation_Code_classing = 'O|P|M'

mycs_data['Occupation_Code_classing'] = np.where(mycs_data['Occupation_Code'] =="Self-employed", 'B', 'O|P|M')

In [489]:
mycs_data.groupby(['Occupation_Code_classing'])['num_target'].mean() # check the good score

Occupation_Code_classing
B        0.948706
O|P|M    0.930364
Name: num_target, dtype: float64

## Time with bank 

In [490]:
# Inspect the column
mycs_data['Time_with_Bank'].value_counts(dropna=False).sort_index()

0       188
1        83
2        62
3        73
4        66
       ... 
4503      1
4510      1
4512      1
4912      1
5501      1
Name: Time_with_Bank, Length: 388, dtype: int64

In [491]:
mycs_data['Time_with_Bank'].describe()

count    17968.000000
mean       717.527549
std        517.706832
min          0.000000
25%        304.000000
50%        702.000000
75%       1112.000000
max       5501.000000
Name: Time_with_Bank, dtype: float64

In [492]:
mycs_data['Time_with_Bank_classing'] = pd.cut(mycs_data['Time_with_Bank'],[0, 500, 1000, 1500, 2000, np.inf],right=False) #set a range

In [493]:
mycs_data['Time_with_Bank_classing'].value_counts().sort_index()

[0.0, 500.0)        6743
[500.0, 1000.0)     4909
[1000.0, 1500.0)    5694
[1500.0, 2000.0)     266
[2000.0, inf)        356
Name: Time_with_Bank_classing, dtype: int64

In [494]:
mycs_data.groupby(['Time_with_Bank_classing'])['num_target'].mean() # check the good score

Time_with_Bank_classing
[0.0, 500.0)        0.901824
[500.0, 1000.0)     0.947647
[1000.0, 1500.0)    0.966807
[1500.0, 2000.0)    0.966165
[2000.0, inf)       0.963483
Name: num_target, dtype: float64

the data is correct since it shows that the customers get better score when they have more years with the bank 

# Creating Dummy Variables

In [495]:
mycs_data.columns #always helpful to see the columns which I'll convert

Index(['Account_Number', 'Account_Type', 'Final_Decision',
       'Current_Delinquency_status', 'Application_Date', 'Application_Score',
       'Cheque_Card_Flag', 'Existing_Customer_Flag', 'Gross_Annual_Income',
       'Home_Telephone_Number', 'Insurance_Required', 'Loan_Amount',
       'Loan_Payment_Frequency', 'Loan_Payment_Method', 'Marital_Status',
       'Number_of_Dependants', 'Number_of_Payments', 'Occupation_Code',
       'Promotion_Type', 'Residential_Status', 'Time_at_Address',
       'Time_in_Employment', 'Time_with_Bank', 'Weight_Factor', 'GB_Flag',
       'Age_of_Applicant', 'Application_Month', 'Bureau_Score',
       'SP_ER_Reference', 'SP_Number_Of_Searches_L6M', 'SP_Number_of_CCJs',
       'loan_to_income', 'split', 'target', 'num_target',
       'loan_to_income_classing', 'Bureau_Score_classing',
       'Occupation_Code_classing', 'Time_with_Bank_classing'],
      dtype='object')

In [496]:
# It's cleaner to create a new data frame, which stores only he variables which we will create the model with
mycs_data_model = mycs_data[['target','num_target','loan_to_income','loan_to_income_classing','Occupation_Code','Occupation_Code_classing',
                         'Bureau_Score' ,'Bureau_Score_classing', 'Time_with_Bank', 'Time_with_Bank_classing']].copy()

In [497]:
#Creating the dummies
mycs_data_model = pd.concat([mycs_data_model,
                    pd.get_dummies(mycs_data_model['loan_to_income_classing'],prefix ='loan_to_income',dummy_na= False),
                    pd.get_dummies(mycs_data_model['Occupation_Code_classing'],prefix ='Occupation_Code',dummy_na= False),
                    pd.get_dummies(mycs_data_model['Bureau_Score_classing'],prefix ='Bureau_Score',dummy_na= False),
                    pd.get_dummies(mycs_data_model['Time_with_Bank_classing'],prefix ='Time_with_Bank',dummy_na= False)], axis=1,ignore_index=False,join='outer')

# The model

In [498]:
mycs_data_model.shape

(17968, 30)

In [499]:
#Create a holdout sample of 30% of the populaiton (test_size = 0.3)
X_train, X_test,y_train,y_test = train_test_split(mycs_data_model,mycs_data_model['num_target'],test_size = 0.3,random_state=42)

In [500]:
#Create a new variable 'sample' in the two dataframes created by the train_test_split
X_train['sample'] = 'Training'
X_test['sample'] = 'Testing'

<ipython-input-500-c393e3ab4559>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['sample'] = 'Training'
<ipython-input-500-c393e3ab4559>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['sample'] = 'Testing'


In [501]:
#Check the shape of X_train
X_train.shape

(12577, 31)

In [502]:
#check the shape of the combined data frames 
mycs_data_model = pd.concat([X_train,X_test],ignore_index=False,sort=False)
mycs_data_model = mycs_data_model.reset_index(drop=True)
mycs_data_model.shape

(17968, 31)

In [503]:
mycs_data_model['sample'].value_counts(dropna=False)

Training    12577
Testing      5391
Name: sample, dtype: int64

In [504]:
mycs_data_model.head(10)

,target,num_target,loan_to_income,loan_to_income_classing,Occupation_Code,Occupation_Code_classing,Bureau_Score,Bureau_Score_classing,Time_with_Bank,Time_with_Bank_classing,...,"Bureau_Score_[710.0, 810.0)","Bureau_Score_[810.0, 910.0)","Bureau_Score_[910.0, 1010.0)","Bureau_Score_[1010.0, inf)","Time_with_Bank_[0.0, 500.0)","Time_with_Bank_[500.0, 1000.0)","Time_with_Bank_[1000.0, 1500.0)","Time_with_Bank_[1500.0, 2000.0)","Time_with_Bank_[2000.0, inf)",sample
0,Good,1,34.01,"[20.0, 40.0)",Pensioner,O|P|M,1002,"[910.0, 1010.0)",407,"[0.0, 500.0)",...,0,0,1,0,1,0,0,0,0,Training
1,Good,1,12.50,"[0.0, 20.0)",Self-employed,B,855,"[810.0, 910.0)",205,"[0.0, 500.0)",...,0,1,0,0,1,0,0,0,0,Training
2,Good,1,-9999997.00,"[-9999997.0, 0.0)",Pensioner,O|P|M,970,"[910.0, 1010.0)",1002,"[1000.0, 1500.0)",...,0,0,1,0,0,0,1,0,0,Training
3,Good,1,18.00,"[0.0, 20.0)",Employee,O|P|M,729,"[710.0, 810.0)",112,"[0.0, 500.0)",...,1,0,0,0,1,0,0,0,0,Training
4,Good,1,45.00,"[40.0, 60.0)",Employee,O|P|M,899,"[810.0, 910.0)",1302,"[1000.0, 1500.0)",...,0,1,0,0,0,0,1,0,0,Training
5,Good,1,11.74,"[0.0, 20.0)",Self-employed,B,1009,"[910.0, 1010.0)",305,"[0.0, 500.0)",...,0,0,1,0,1,0,0,0,0,Training
6,Good,1,128.89,"[100.0, inf)",Employee,O|P|M,961,"[910.0, 1010.0)",1301,"[1000.0, 1500.0)",...,0,0,1,0,0,0,1,0,0,Training
7,Good,1,46.15,"[40.0, 60.0)",Self-employed,B,1031,"[1010.0, inf)",1201,"[1000.0, 1500.0)",...,0,0,0,1,0,0,1,0,0,Training
8,Good,1,68.00,"[60.0, 80.0)",Self-employed,B,1002,"[910.0, 1010.0)",1002,"[1000.0, 1500.0)",...,0,0,1,0,0,0,1,0,0,Training
9,Good,1,13.54,"[0.0, 20.0)",Employee,O|P|M,946,"[910.0, 1010.0)",1205,"[1000.0, 1500.0)",...,0,0,1,0,0,0,1,0,0,Training


In [505]:
mycs_data_model.columns

Index(['target', 'num_target', 'loan_to_income', 'loan_to_income_classing',
       'Occupation_Code', 'Occupation_Code_classing', 'Bureau_Score',
       'Bureau_Score_classing', 'Time_with_Bank', 'Time_with_Bank_classing',
       'loan_to_income_[-9999998.0, -9999997.0)',
       'loan_to_income_[-9999997.0, 0.0)', 'loan_to_income_[0.0, 20.0)',
       'loan_to_income_[20.0, 40.0)', 'loan_to_income_[40.0, 60.0)',
       'loan_to_income_[60.0, 80.0)', 'loan_to_income_[80.0, 100.0)',
       'loan_to_income_[100.0, inf)', 'Occupation_Code_B',
       'Occupation_Code_O|P|M', 'Bureau_Score_[-inf, 710.0)',
       'Bureau_Score_[710.0, 810.0)', 'Bureau_Score_[810.0, 910.0)',
       'Bureau_Score_[910.0, 1010.0)', 'Bureau_Score_[1010.0, inf)',
       'Time_with_Bank_[0.0, 500.0)', 'Time_with_Bank_[500.0, 1000.0)',
       'Time_with_Bank_[1000.0, 1500.0)', 'Time_with_Bank_[1500.0, 2000.0)',
       'Time_with_Bank_[2000.0, inf)', 'sample'],
      dtype='object')

In [506]:
#Define the X and y parameters needed for running the model. 
#X will contain all predictors/dummies from the Training subpopulation. y will contain the respective target variable
#In order to avoin perfect correlation, we must select a "null band" which will not be used as a predictor. 
#That is why one band is missing from each of the 3 variables
y = mycs_data_model[mycs_data_model['sample']=='Training']['num_target']
X = mycs_data_model[mycs_data_model['sample']=='Training'][['loan_to_income_[0.0, 20.0)',
                                                            'loan_to_income_[20.0, 40.0)', 'loan_to_income_[40.0, 60.0)',
                                                            'loan_to_income_[60.0, 80.0)', 'loan_to_income_[80.0, 100.0)',
                                                            'loan_to_income_[100.0, inf)',
                                                            'Occupation_Code_B', 'Occupation_Code_O|P|M',
                                                            'Bureau_Score_[-inf, 710.0)', 'Bureau_Score_[710.0, 810.0)',
                                                            'Bureau_Score_[810.0, 910.0)', 'Bureau_Score_[910.0, 1010.0)',
                                                            'Bureau_Score_[1010.0, inf)', 'Time_with_Bank_[0.0, 500.0)',
                                                            'Time_with_Bank_[500.0, 1000.0)', 'Time_with_Bank_[1000.0, 1500.0)',
                                                            'Time_with_Bank_[1500.0, 2000.0)', 'Time_with_Bank_[2000.0, inf)']]

In [507]:
#Define the model parameters and fit it to the data, print a summary of the model
X = sm.tools.tools.add_constant(X)
model_1 = sm.GLM(y,X,family=sm.genmod.families.Binomial(link=sm.genmod.families.links.logit))
model_final = model_1.fit()
print(model_final.summary())

<ipython-input-507-2edf027e184d>:3: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  model_1 = sm.GLM(y,X,family=sm.genmod.families.Binomial(link=sm.genmod.families.links.logit))


                 Generalized Linear Model Regression Results                  
Dep. Variable:             num_target   No. Observations:                12577
Model:                            GLM   Df Residuals:                    12561
Model Family:                Binomial   Df Model:                           15
Link Function:                  logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2803.8
Date:                Fri, 02 Jul 2021   Deviance:                       5607.7
Time:                        10:23:30   Pearson chi2:                 1.25e+04
No. Iterations:                   100                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [508]:
#the model is saved as python object
model_final

In [509]:
# for the prediction:
X['Prediction'] = model_final.predict()

In [510]:
# let's check the prediction
X

,const,"loan_to_income_[0.0, 20.0)","loan_to_income_[20.0, 40.0)","loan_to_income_[40.0, 60.0)","loan_to_income_[60.0, 80.0)","loan_to_income_[80.0, 100.0)","loan_to_income_[100.0, inf)",Occupation_Code_B,Occupation_Code_O|P|M,"Bureau_Score_[-inf, 710.0)","Bureau_Score_[710.0, 810.0)","Bureau_Score_[810.0, 910.0)","Bureau_Score_[910.0, 1010.0)","Bureau_Score_[1010.0, inf)","Time_with_Bank_[0.0, 500.0)","Time_with_Bank_[500.0, 1000.0)","Time_with_Bank_[1000.0, 1500.0)","Time_with_Bank_[1500.0, 2000.0)","Time_with_Bank_[2000.0, inf)",Prediction
0,1.0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0.947895
1,1.0,1,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0.902471
2,1.0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0.978590
3,1.0,1,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0.794047
4,1.0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0.913936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12572,1.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0.957824
12573,1.0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0.957360
12574,1.0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0.889277
12575,1.0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0.858397


In [511]:
y.mean()

0.9349606424425538

In [512]:
#Save as pickle file
mycs_data_model.to_pickle(project_path + r"\loan_model.pkl")

In [513]:
mycs_data_model.columns

Index(['target', 'num_target', 'loan_to_income', 'loan_to_income_classing',
       'Occupation_Code', 'Occupation_Code_classing', 'Bureau_Score',
       'Bureau_Score_classing', 'Time_with_Bank', 'Time_with_Bank_classing',
       'loan_to_income_[-9999998.0, -9999997.0)',
       'loan_to_income_[-9999997.0, 0.0)', 'loan_to_income_[0.0, 20.0)',
       'loan_to_income_[20.0, 40.0)', 'loan_to_income_[40.0, 60.0)',
       'loan_to_income_[60.0, 80.0)', 'loan_to_income_[80.0, 100.0)',
       'loan_to_income_[100.0, inf)', 'Occupation_Code_B',
       'Occupation_Code_O|P|M', 'Bureau_Score_[-inf, 710.0)',
       'Bureau_Score_[710.0, 810.0)', 'Bureau_Score_[810.0, 910.0)',
       'Bureau_Score_[910.0, 1010.0)', 'Bureau_Score_[1010.0, inf)',
       'Time_with_Bank_[0.0, 500.0)', 'Time_with_Bank_[500.0, 1000.0)',
       'Time_with_Bank_[1000.0, 1500.0)', 'Time_with_Bank_[1500.0, 2000.0)',
       'Time_with_Bank_[2000.0, inf)', 'sample'],
      dtype='object')

In [514]:
mycs_data_model.head()

,target,num_target,loan_to_income,loan_to_income_classing,Occupation_Code,Occupation_Code_classing,Bureau_Score,Bureau_Score_classing,Time_with_Bank,Time_with_Bank_classing,...,"Bureau_Score_[710.0, 810.0)","Bureau_Score_[810.0, 910.0)","Bureau_Score_[910.0, 1010.0)","Bureau_Score_[1010.0, inf)","Time_with_Bank_[0.0, 500.0)","Time_with_Bank_[500.0, 1000.0)","Time_with_Bank_[1000.0, 1500.0)","Time_with_Bank_[1500.0, 2000.0)","Time_with_Bank_[2000.0, inf)",sample
0,Good,1,34.01,"[20.0, 40.0)",Pensioner,O|P|M,1002,"[910.0, 1010.0)",407,"[0.0, 500.0)",...,0,0,1,0,1,0,0,0,0,Training
1,Good,1,12.50,"[0.0, 20.0)",Self-employed,B,855,"[810.0, 910.0)",205,"[0.0, 500.0)",...,0,1,0,0,1,0,0,0,0,Training
2,Good,1,-9999997.00,"[-9999997.0, 0.0)",Pensioner,O|P|M,970,"[910.0, 1010.0)",1002,"[1000.0, 1500.0)",...,0,0,1,0,0,0,1,0,0,Training
3,Good,1,18.00,"[0.0, 20.0)",Employee,O|P|M,729,"[710.0, 810.0)",112,"[0.0, 500.0)",...,1,0,0,0,1,0,0,0,0,Training
4,Good,1,45.00,"[40.0, 60.0)",Employee,O|P|M,899,"[810.0, 910.0)",1302,"[1000.0, 1500.0)",...,0,1,0,0,0,0,1,0,0,Training


In [515]:
X.columns.values

array(['const', 'loan_to_income_[0.0, 20.0)',
       'loan_to_income_[20.0, 40.0)', 'loan_to_income_[40.0, 60.0)',
       'loan_to_income_[60.0, 80.0)', 'loan_to_income_[80.0, 100.0)',
       'loan_to_income_[100.0, inf)', 'Occupation_Code_B',
       'Occupation_Code_O|P|M', 'Bureau_Score_[-inf, 710.0)',
       'Bureau_Score_[710.0, 810.0)', 'Bureau_Score_[810.0, 910.0)',
       'Bureau_Score_[910.0, 1010.0)', 'Bureau_Score_[1010.0, inf)',
       'Time_with_Bank_[0.0, 500.0)', 'Time_with_Bank_[500.0, 1000.0)',
       'Time_with_Bank_[1000.0, 1500.0)',
       'Time_with_Bank_[1500.0, 2000.0)', 'Time_with_Bank_[2000.0, inf)',
       'Prediction'], dtype=object)

In [516]:
#new dictionary for the dummy and its coefficient from the model
model_coefficients = {
            "loan_to_income_[0.0, 20.0)" : -0.3865,
            "loan_to_income_[20.0, 40.0)" : -0.5167,
            "loan_to_income_[40.0, 60.0)" : -0.5618,
            "loan_to_income_[60.0, 80.0)" : -0.7347, 
            "loan_to_income_[80.0, 100.0)" : -0.5484,
            "loan_to_income_[100.0, inf)" : -0.7980,
            "Occupation_Code_B" : 0.7513, 
            "Occupation_Code_O|P|M" : 0.6597,
            "Bureau_Score_[-inf, 710.0)" : -1.4975,
            "Bureau_Score_[710.0, 810.0)" : -0.4761,
            "Bureau_Score_[810.0, 910.0)" : 0.3078,
            "Bureau_Score_[910.0, 1010.0)" : 1.2056, 
            "Bureau_Score_[1010.0, inf)" : 1.8711,
            "Time_with_Bank_[0.0, 500.0)" : 0.1414, 
            "Time_with_Bank_[500.0, 1000.0)" : 0.2216,
            "Time_with_Bank_[1000.0, 1500.0)" : 0.5460,
            "Time_with_Bank_[1500.0, 2000.0)" : 0.3370, 
            "Time_with_Bank_[2000.0, inf)" : 0.1651
            }

In [517]:
#Define a function that takes the dataframe, value of the intercept from the model and 
#     the dictionary of dummy - coefficient and creates a new variable - score

def score(df, intercept, coefficients):
    value_to_add = 0
   
    for name, coefficient in coefficients.items():
        value_to_add += coefficient*(df[name])
       
    df["Score"] = round((intercept + value_to_add)*100,0)

In [518]:
#Execute the function on the full dataframe containing both Training and Testing populations
score(mycs_data_model, 1.4110, model_coefficients)

# Score Distribution Report

In [519]:
mycs_data_model['Score']

0        290.0
1        222.0
2        382.0
3        135.0
4        236.0
         ...  
17963    307.0
17964    221.0
17965    344.0
17966    277.0
17967    344.0
Name: Score, Length: 17968, dtype: float64

In [520]:
#Split the dataframe into Training (aka Development) and Testing(aka Validation)
mycs_data_model_development = mycs_data_model[mycs_data_model['sample'] == 'Training']
mycs_data_model_validation = mycs_data_model[mycs_data_model['sample'] == 'Testing']

In [521]:
#We check the value at each 5th percentile
for i in range(0,101,5):
    print(np.percentile(mycs_data_model_development['Score'],i), ",")

-8.0 ,
172.0 ,
196.0 ,
205.0 ,
213.0 ,
222.0 ,
254.0 ,
277.0 ,
292.0 ,
298.0 ,
303.0 ,
307.0 ,
312.0 ,
326.0 ,
331.0 ,
344.0 ,
351.0 ,
369.0 ,
382.0 ,
406.0 ,
458.0 ,


In [522]:
mycs_data_model_development['Score'].describe()

count    12577.000000
mean       290.831041
std         73.805662
min         -8.000000
25%        222.000000
50%        303.000000
75%        344.000000
max        458.000000
Name: Score, dtype: float64

In [523]:
#We create a crosstable of the score binned at 5% and the target variable 
pd.crosstab(pd.cut(mycs_data_model_development['Score'],bins=[
                                                            -8.0 ,
                                                            172.0 ,
                                                            196.0 ,
                                                            205.0 ,
                                                            213.0 ,
                                                            222.0 ,                                     
                                                            254.0 ,
                                                            277.0 ,
                                                            292.0 ,
                                                            298.0 ,
                                                            303.0 ,
                                                            307.0 ,
                                                            312.0 ,
                                                            326.0 ,
                                                            331.0 ,
                                                            344.0 ,
                                                            351.0 ,
                                                            369.0 ,
                                                            382.0 ,
                                                            406.0 ,
                                                            458.0 ]), mycs_data_model_development["target"])

target,Bad,Good
Score,,
"(-8.0, 172.0]",135,547
"(172.0, 196.0]",88,516
"(196.0, 205.0]",74,550
"(205.0, 213.0]",106,880
"(213.0, 222.0]",53,420
"(222.0, 254.0]",28,439
"(254.0, 277.0]",43,545
"(277.0, 292.0]",49,585
"(292.0, 298.0]",34,723


In [524]:
score_distribution_record = pd.crosstab(pd.cut(mycs_data_model_development['Score'],bins=[
                                                            -8.0 ,
                                                            172.0 ,
                                                            196.0 ,
                                                            205.0 ,
                                                            213.0 ,
                                                            222.0 ,                                     
                                                            254.0 ,
                                                            277.0 ,
                                                            292.0 ,
                                                            298.0 ,
                                                            303.0 ,
                                                            307.0 ,
                                                            312.0 ,
                                                            326.0 ,
                                                            331.0 ,
                                                            344.0 ,
                                                            351.0 ,
                                                            369.0 ,
                                                            382.0 ,
                                                            406.0 ,
                                                            458.0 ]), mycs_data_model_development["target"])

In [525]:
#Calculate the Bad rate at each row, present it as a percent in format NN.NN%
score_distribution_record['Bad Rate'] = round((score_distribution_record["Bad"]/(score_distribution_record["Bad"] + score_distribution_record["Good"]))*100,2)
#Calculate the Good to Bad rate at each row, showing how many Good accounts we have for 1 Bad account
score_distribution_record['GB Odds'] = round(score_distribution_record["Good"]/score_distribution_record["Bad"],2)
#Calculate the percent of the total population at each row
score_distribution_record['% Total'] = round((score_distribution_record["Bad"] + score_distribution_record["Good"])/(score_distribution_record["Bad"].sum() + score_distribution_record["Good"].sum())*100,2)
score_distribution_record

target,Bad,Good,Bad Rate,GB Odds,% Total
Score,,,,,
"(-8.0, 172.0]",135,547,19.79,4.05,5.42
"(172.0, 196.0]",88,516,14.57,5.86,4.80
"(196.0, 205.0]",74,550,11.86,7.43,4.96
"(205.0, 213.0]",106,880,10.75,8.30,7.84
"(213.0, 222.0]",53,420,11.21,7.92,3.76
"(222.0, 254.0]",28,439,6.00,15.68,3.71
"(254.0, 277.0]",43,545,7.31,12.67,4.68
"(277.0, 292.0]",49,585,7.73,11.94,5.04
"(292.0, 298.0]",34,723,4.49,21.26,6.02


On the base of this distribution report the thresholds for Rejecting, Referring and Accepting a customer will be such:
- Reject - All customers with score less than or equal 292 points
- Refer for further manual assessment - between 293 points (including), and 312 points (including)
- Accept - All customers with score greater or equal to 313 points. 

In [526]:
mycs_data_model.columns

Index(['target', 'num_target', 'loan_to_income', 'loan_to_income_classing',
       'Occupation_Code', 'Occupation_Code_classing', 'Bureau_Score',
       'Bureau_Score_classing', 'Time_with_Bank', 'Time_with_Bank_classing',
       'loan_to_income_[-9999998.0, -9999997.0)',
       'loan_to_income_[-9999997.0, 0.0)', 'loan_to_income_[0.0, 20.0)',
       'loan_to_income_[20.0, 40.0)', 'loan_to_income_[40.0, 60.0)',
       'loan_to_income_[60.0, 80.0)', 'loan_to_income_[80.0, 100.0)',
       'loan_to_income_[100.0, inf)', 'Occupation_Code_B',
       'Occupation_Code_O|P|M', 'Bureau_Score_[-inf, 710.0)',
       'Bureau_Score_[710.0, 810.0)', 'Bureau_Score_[810.0, 910.0)',
       'Bureau_Score_[910.0, 1010.0)', 'Bureau_Score_[1010.0, inf)',
       'Time_with_Bank_[0.0, 500.0)', 'Time_with_Bank_[500.0, 1000.0)',
       'Time_with_Bank_[1000.0, 1500.0)', 'Time_with_Bank_[1500.0, 2000.0)',
       'Time_with_Bank_[2000.0, inf)', 'sample', 'Score'],
      dtype='object')

# Validation

We need to know if the model will perform as well on new data as it does on the data we used to create it. To do that we must use Kolmogorov-Smirnov test.
 

In [527]:
mycs_data_model['sample'].value_counts(dropna=False)

Training    12577
Testing      5391
Name: sample, dtype: int64

Since we already created a 5% sdr, we can reuse it

In [528]:
validation_development = pd.crosstab(pd.cut(mycs_data_model_development['Score'],bins=[
                                                            -8.0 ,
                                                            172.0 ,
                                                            196.0 ,
                                                            205.0 ,
                                                            213.0 ,
                                                            222.0 ,                                     
                                                            254.0 ,
                                                            277.0 ,
                                                            292.0 ,
                                                            298.0 ,
                                                            303.0 ,
                                                            307.0 ,
                                                            312.0 ,
                                                            326.0 ,
                                                            331.0 ,
                                                            344.0 ,
                                                            351.0 ,
                                                            369.0 ,
                                                            382.0 ,
                                                            406.0 ,
                                                            458.0 ]), mycs_data_model_development["target"])

In [529]:
validation_validation = pd.crosstab(pd.cut(mycs_data_model_development['Score'],bins=[
                                                            -8.0 ,
                                                            172.0 ,
                                                            196.0 ,
                                                            205.0 ,
                                                            213.0 ,
                                                            222.0 ,                                     
                                                            254.0 ,
                                                            277.0 ,
                                                            292.0 ,
                                                            298.0 ,
                                                            303.0 ,
                                                            307.0 ,
                                                            312.0 ,
                                                            326.0 ,
                                                            331.0 ,
                                                            344.0 ,
                                                            351.0 ,
                                                            369.0 ,
                                                            382.0 ,
                                                            406.0 ,
                                                            458.0 ]), mycs_data_model_development["target"])

In [530]:
validation_total = pd.crosstab(pd.cut(mycs_data_model_development['Score'],bins=[
                                                            -8.0 ,
                                                            172.0 ,
                                                            196.0 ,
                                                            205.0 ,
                                                            213.0 ,
                                                            222.0 ,                                     
                                                            254.0 ,
                                                            277.0 ,
                                                            292.0 ,
                                                            298.0 ,
                                                            303.0 ,
                                                            307.0 ,
                                                            312.0 ,
                                                            326.0 ,
                                                            331.0 ,
                                                            344.0 ,
                                                            351.0 ,
                                                            369.0 ,
                                                            382.0 ,
                                                            406.0 ,
                                                            458.0 ]), mycs_data_model_development["target"])

In [531]:
validation_total.reset_index(inplace = True)
validation_total.columns.name = 'index'
validation_development.reset_index(inplace = True)
validation_development.columns.name = 'index'
validation_validation.reset_index(inplace = True)
validation_validation.columns.name = 'index'

In [532]:
validation_total['total'] = validation_total["Bad"] + validation_total["Good"]
validation_development['total'] = validation_development["Bad"] + validation_development["Good"]
validation_validation['total'] = validation_validation["Bad"]  + validation_validation["Good"]

In [533]:
validation_total

index,Score,Bad,Good,total
0,"(-8.0, 172.0]",135,547,682
1,"(172.0, 196.0]",88,516,604
2,"(196.0, 205.0]",74,550,624
3,"(205.0, 213.0]",106,880,986
4,"(213.0, 222.0]",53,420,473
5,"(222.0, 254.0]",28,439,467
6,"(254.0, 277.0]",43,545,588
7,"(277.0, 292.0]",49,585,634
8,"(292.0, 298.0]",34,723,757
9,"(298.0, 303.0]",34,748,782


In [534]:
data_frame_combined = validation_development.merge(right=validation_validation, on='Score').merge(right=validation_total, on='Score')
data_frame_combined

index,Score,Bad_x,Good_x,total_x,Bad_y,Good_y,total_y,Bad,Good,total
0,"(-8.0, 172.0]",135,547,682,135,547,682,135,547,682
1,"(172.0, 196.0]",88,516,604,88,516,604,88,516,604
2,"(196.0, 205.0]",74,550,624,74,550,624,74,550,624
3,"(205.0, 213.0]",106,880,986,106,880,986,106,880,986
4,"(213.0, 222.0]",53,420,473,53,420,473,53,420,473
5,"(222.0, 254.0]",28,439,467,28,439,467,28,439,467
6,"(254.0, 277.0]",43,545,588,43,545,588,43,545,588
7,"(277.0, 292.0]",49,585,634,49,585,634,49,585,634
8,"(292.0, 298.0]",34,723,757,34,723,757,34,723,757
9,"(298.0, 303.0]",34,748,782,34,748,782,34,748,782


In [535]:
data_frame_combined.columns

Index(['Score', 'Bad_x', 'Good_x', 'total_x', 'Bad_y', 'Good_y', 'total_y',
       'Bad', 'Good', 'total'],
      dtype='object', name='index')

In [536]:
renaming = {'Bad_x': 'bad_dev', 
            'Bad_y': 'bad_val', 
            'Good_x': 'good_dev', 
            'Good_y': 'good_val', 
            'Bad': 'bad_all', 
            'Good': 'good_all',
            'total_x': 'total_dev',
            'total_y': 'total_val',
            'total': 'total_all'}

In [537]:
data_frame_combined = data_frame_combined.rename(columns = renaming)
data_frame_combined

index,Score,bad_dev,good_dev,total_dev,bad_val,good_val,total_val,bad_all,good_all,total_all
0,"(-8.0, 172.0]",135,547,682,135,547,682,135,547,682
1,"(172.0, 196.0]",88,516,604,88,516,604,88,516,604
2,"(196.0, 205.0]",74,550,624,74,550,624,74,550,624
3,"(205.0, 213.0]",106,880,986,106,880,986,106,880,986
4,"(213.0, 222.0]",53,420,473,53,420,473,53,420,473
5,"(222.0, 254.0]",28,439,467,28,439,467,28,439,467
6,"(254.0, 277.0]",43,545,588,43,545,588,43,545,588
7,"(277.0, 292.0]",49,585,634,49,585,634,49,585,634
8,"(292.0, 298.0]",34,723,757,34,723,757,34,723,757
9,"(298.0, 303.0]",34,748,782,34,748,782,34,748,782
